In [1]:
import pandas as pd
import datetime
import os
import numpy as np
print(datetime.datetime.now())

2020-05-07 13:50:50.145042


In [2]:
file_excel=pd.ExcelFile("./BL_zips_for_2019_Quad_3_Stores_JL_2020-05-04.xlsx")
df_zip_by_store=file_excel.parse("zips_for_quad_3_only",dtype=str)

In [13]:
file_excel.sheet_names

['zips_for_quad_3_only']

In [5]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200501-152115-660.txt",
                           dtype=str,sep="|")
df_store_list=df_store_list[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd']]
for col in df_store_list.columns.tolist()[1:]:
    df_store_list=df_store_list.rename(columns={col:"store_"+col})
    
df_store_list['store_zip_cd']=df_store_list['store_zip_cd'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5))

In [6]:
df_store_list=df_store_list[df_store_list['location_id'].isin(df_zip_by_store['location_id'].unique().tolist())]
print(df_store_list.shape,df_zip_by_store['location_id'].nunique())

(82, 6) 82


In [9]:
df_DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2]).drop_duplicates()
df_DMA_zip.columns=["store_zip_cd","DMA"]
df_DMA_zip_list=df_DMA_zip.groupby("store_zip_cd")['DMA'].apply(list).to_frame().reset_index()
df_DMA_zip_list=df_DMA_zip_list.rename(columns={"DMA":"DMA_list"})

In [10]:
df_store_list=pd.merge(df_store_list,df_DMA_zip_list,on="store_zip_cd",how="left")
df_store_list.head(2)

,location_id,store_address_line_1,store_address_line_2,store_city_nm,store_state_nm,store_zip_cd,DMA_list
0,132,2050 S 22ND ST,NaN,LAFAYETTE,IN,47905,"[LAFAYETTE, IN]"
1,209,1907 S 11TH ST,NaN,NILES,MI,49120,[SOUTH BEND-ELKHART]


In [11]:
df_store_list['DMA_1']=df_store_list['DMA_list'].apply(lambda x: x[0])
df_store_list['DMA_2']=df_store_list['DMA_list'].apply(lambda x: x[1] if len(x)==2 else np.nan)


In [12]:
df_output_detail=df_zip_by_store[['location_id','open_dt','latitude_meas','longitude_meas','zip_cd','zip_type']]
df_output_detail=pd.merge(df_output_detail,df_store_list,on="location_id",how="left")
df_output_detail.head(2)

,location_id,open_dt,latitude_meas,longitude_meas,zip_cd,zip_type,store_address_line_1,store_address_line_2,store_city_nm,store_state_nm,store_zip_cd,DMA_list,DMA_1,DMA_2
0,132,1986-02-21,40.397225,-86.871866,47909,P,2050 S 22ND ST,NaN,LAFAYETTE,IN,47905,"[LAFAYETTE, IN]","LAFAYETTE, IN",NaN
1,132,1986-02-21,40.397225,-86.871866,47905,P,2050 S 22ND ST,NaN,LAFAYETTE,IN,47905,"[LAFAYETTE, IN]","LAFAYETTE, IN",NaN


In [14]:
df_1_unique_zip_cd=df_output_detail[['zip_cd','DMA_1','DMA_2']].drop_duplicates()
df_1_unique_zip_cd.shape,df_1_unique_zip_cd['zip_cd'].nunique()

((412, 3), 412)

In [17]:
writer=pd.ExcelWriter("./BL_zips_for_2019_Quad_3_Stores_with_DMA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_1_unique_zip_cd.to_excel(writer,"unique_zip_with_DMA",index=False)
df_output_detail.to_excel(writer,"detail_by_store",index=False)
df_store_list.to_excel(writer,"store_info",index=False)
writer.save()
